## Using OperationDefinition to generate Observation instances

See discussion on [Zulip](https://chat.fhir.org/#narrow/stream/179256-Orders-and.20Observation.20WG/topic/ObservationDefinition.20does.20not.20work.20for.20the.20simple.20stuff) regarding the common and simple need for a way to use [ObservationDefinition](http://build.fhir.org/observationdefinition.html) as table or spreadsheet to create Observation instances based on a few 'base' profiles.ObservationDefinition needs to be structured simply and flat enough to be able to directly transform into a table to contain the stuff you need for creating an Observation.

The following proof of concept Python script demonstrates this.  It takes test data (i.e. observationdefinition) in the form of a CSV file and applies them to common Observation template to generates Observation instances.

The table is like a Dictionary of Observations for use and is a shortcut to creating 100s or 1000s of individual profiles.


run in python 3.6


Outline:
    
1. read row from od spreadsheets as dict
2. apply od spreadsheet data to named profile (SD)
3. write/display profile



In [143]:
# imports

import json, os, sys, csv, datetime
from collections import namedtuple
from pandas import read_csv
from datetime import datetime
from json import dumps
from requests import post
from IPython.display import display, Markdown


In [176]:
# global variables and templates

in_file = '/Users/ehaas/Documents/Python/Notebooks/OD - od.csv'
sheet_name = 'od'
now = f'{datetime.utcnow().isoformat()}Z'
outfile = 'outfile'  # name of output file
headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}
fhir_test_server = 'http://test.fhir.org/r4'

In [177]:
# preview data from csv file

df = read_csv(in_file, encoding = "ISO-8859-1")
# to convert to named tuple
#Data = namedtuple('Data', df.columns)
#data = [Data(*r[1:]) for r in df.itertuples() if r[0] > 0]

df

,Index,Name,Description,Profile,code,codeCoding_0_Code,codeCoding_0_Display,codeText,valueType,unit,...,appliesTo,age,refString,refLow,refHigh,interp,interpDescription,interpCode,interpLowDecimal,interpHighDecimal
0,row number for OD item when using a spreadsheet,The User friendly name of OD can be used to li...,NaN,"The SD that this OD applies to, i.e., a lab nu...",code(CodeableConcept) to apply to profile,NaN,NaN,NaN,valueType(Quantity|codeableConcept|string|äó_)...,Unit(Coding) to apply to profile if is Quantity,...,Corresponding population this applies to in O,NaN,RR string value - only Range and string are re...,RR Range.low and RR Range.high unit string val...,RR Range.low and RR Range.high unit string val...,interpretation 0..* is a description of the lo...,text based description of how to get at partic...,the corresponding interpretation code in O (Hi...,the values that correspond to the interpretati...,the value that correspond to the high end of t...
1,1,bg,Blood Glucose OperationDefinition,http://hl7.org/fhir/us/core/StructureDefinitio...,(expanded out to primitives-->),2339-0,Glucose Bld-mCnc,http://loinc.org,NaN,(expanded out to primitives-->),...,NaN,NaN,NaN,70,99,NaN,NaN,NaN,NaN,NaN
2,2,bun,BUN OperationDefinition,http://hl7.org/fhir/us/core/StructureDefinitio...,(expanded out to primitives-->),3094-0,BUN SerPl-mCnc,http://loinc.org,NaN,(expanded out to primitives-->),...,NaN,NaN,NaN,7,20,NaN,NaN,NaN,NaN,NaN
3,3,na,Serum Sodium OperationDefinition,http://hl7.org/fhir/us/core/StructureDefinitio...,(expanded out to primitives-->),2951-2,Sodium SerPl-sCnc,http://loinc.org,NaN,(expanded out to primitives-->),...,NaN,NaN,NaN,135,145,NaN,NaN,NaN,NaN,NaN


### Function to fetch data from csv file

In [171]:
# dataframes are a pain so will reload as a csv and
# use named tuple structure
def get_testinfo():
    with open(in_file, encoding = "ISO-8859-1") as f:
        reader = csv.reader(f)
        top_row = next(reader)
        # top_row = [t.lower().split(' ') for t in top_row]
        # top_row = ['_'.join(t) for t in top_row]

        Data = namedtuple("Data", top_row)
        next(reader) # skip 2nd row of definitions
        data = [Data(*r) for r in reader]
        return data

    # using named tuple structure make getting data all nice n pretty like
    # allow one to use dot notation like 'data[0].name'

    #print(f'name = {data[0].Name}\
    #      \ndescription = {data[0].Description}\
    #      \ncode = {data[0].codeCoding_0_Code}')


### Function to Populate Observation template (profile instance) with data and return Observation instance

In [169]:
def get_obs(t, p, r, s):# apply od spreadsheet data , patient info, results and status to profile 
    value, effective = r
    return {
      "resourceType": "Observation",
      "meta": { "profile": ["http://hl7.org/fhir/us/core/StructureDefinition/us-core-observationresults"],
      },
      "status": s,
      "category": [{
        "coding": [
          {
            "system": "http://hl7.org/fhir/observation-category",
            "code": "laboratory",
            "display": "Laboratory"
          }
        ],
        "text": "Laboratory"
      }],
      "code": {
        "coding": [
          {
            "system": "http://loinc.org",
            "code": t.codeCoding_0_Code,
            "display": t.codeCoding_0_Display
          }
        ],
        "text": t.codeText
      },
      "subject": {
        "reference": f"Patient/{p.id}",
          "display": p.name
      },
      "effectiveDateTime": effective,
      "valueQuantity": {
        "value": value,
        "unit": t.unitUnit,
        "system": t.unitCodeSystem,
        "code": t.unitCodeCode
          },
      "referenceRange": [
        {
          "low": {
            "value": float(t.refLow),
            "unit": t.unitUnit,
            "system": t.unitCodeSystem,
            "code": t.unitCodeCode
          },
          "high": {
            "value": float(t.refHigh),
            "unit": t.unitUnit,
            "system": t.unitCodeSystem,
            "code": t.unitCodeCode
          },
          "type": {
            "coding": [
              {
                "system": "http://hl7.org/fhir/referencerange-meaning",
                "code": t.refType,
              }
            ]
           }
         }
       ]     
     }




### Gather patient, result values and test=ObservationDefinition data and create Observations

In [172]:

obs_list=[]
# get patient (just a namedtuple for this demo instead of FHIR object)
Patient = namedtuple('Patient','id name')
patient = Patient('1234','Jane Doe')


# get test result data - (just a dictionary or hash for this example)
# k = test name : v = (result value, timestamp)
results = dict(
    bg = (96.0, now),
    bun = (20.0, now),
    na = (135.0, now)
    ) 
print('get test information from od spreadsheets')
tests = get_testinfo()

for t in tests:
    new_obs = get_obs(t=t, p=patient, r=results[t.Name], s='preliminary' )
    print('='*79,'\n','='*30,f'{patient.name} {t.Name} results','='*30,'\n','='*79)
    print(f'Observation = {dumps(new_obs,indent = 3)}')
    obs_list.append(new_obs)



get test information from od spreadsheets
 ============================== Jane Doe bg results ============================== 
Observation = {
   "resourceType": "Observation",
   "meta": {
      "profile": [
         "http://hl7.org/fhir/us/core/StructureDefinition/us-core-observationresults"
      ]
   },
   "status": "preliminary",
   "category": [
      {
         "coding": [
            {
               "system": "http://hl7.org/fhir/observation-category",
               "code": "laboratory",
               "display": "Laboratory"
            }
         ],
         "text": "Laboratory"
      }
   ],
   "code": {
      "coding": [
         {
            "system": "http://loinc.org",
            "code": "2339-0",
            "display": "Glucose Bld-mCnc"
         }
      ],
      "text": "http://loinc.org"
   },
   "subject": {
      "reference": "Patient/1234",
      "display": "Jane Doe"
   },
   "effectiveDateTime": "2019-01-15T07:35:52.747834Z",
   "valueQuantity": {
      "value


###  Validate  A Resource

Using the [$validate](http://build.fhir.org/resource-operation-validate.html) operation, the example is validated 
by a FHIR Reference Server.  The results are displayed below in the human readable text as xhtml.


In [175]:

    # validate using requests
    r = post(f'{fhir_test_server}/Observation/$validate',headers = headers, data = dumps(obs_list[0]))
    # print(r.status_code)
    # view  output
    display(Markdown(f'<h1>Validation output</h1>{r.json()["text"]["div"]}'))


 2019-01-15 01:41:11,947 - DEBUG- Starting new HTTP connection (1): test.fhir.org:80
 2019-01-15 01:41:12,065 - DEBUG- http://test.fhir.org:80 "POST /r4/Observation/$validate HTTP/1.1" 200 2673


<h1>Validation output</h1><div><p><b>Operation Outcome for :Validate resource </b></p><table class="grid"><tr><td><b>Severity</b></td><td><b>Location</b></td><td><b>Details</b></td><td><b>Diagnostics</b></td><td><b>Type</b></td></tr><tr><td>warning</td><td/><td>StructureDefinition reference "http://hl7.org/fhir/us/core/StructureDefinition/us-core-observationresults" could not be resolved</td><td/><td>invalid</td></tr><tr><td>warning</td><td/><td>A resource should have narrative for robust management () text.div.exists()</td><td/><td>invariant</td></tr><tr><td>warning</td><td/><td>ValueSet http://hl7.org/fhir/ValueSet/observation-status|4.0.0 not found</td><td/><td>code-invalid</td></tr><tr><td>information</td><td/><td>None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/observation-category (http://hl7.org/fhir/ValueSet/observation-category, and a code is recommended to come from this value set</td><td/><td>code-invalid</td></tr><tr><td>information</td><td/><td>None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/referencerange-meaning (http://hl7.org/fhir/ValueSet/referencerange-meaning, and a code is recommended to come from this value set</td><td/><td>code-invalid</td></tr></table></div>